In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

import requests
import json

In [4]:
characters = ['Monica Geller', 'Joey Tribbiani', 'Chandler Bing', 'Phoebe Buffay', 'Ross Geller', 'Rachel Green', 'others']

#test set 1 = season 8

with open('sets/test_set1.json') as f:
    test_set1 = json.load(f)

with open('sets/train_set1.json') as f:
    train_set1 = json.load(f)

#CHANGE HERE
test_set = test_set1
train_set = train_set1

In [7]:
#get highest x sentences for each character
def get_highest(tf_idfs, x):
    highest = []
    for tf_idf in tf_idfs:
        highest.append(sorted(tf_idf.data)[-x:])
    return highest


#find sentences in test set that are similar to highest
def find_similar(tf_idfs, highest, x):
    similar = []
    for i in range(len(tf_idfs)):
        similar.append([])
        for j in range(len(tf_idfs[i].data)):
            if tf_idfs[i].data[j] in highest[i]:
                similar[i].append(tf_idfs[i].indices[j])
    return similar

In [5]:
all_lines = []

#corpus
for line in train_set:
    all_lines.append(line['transcript'])

vectorizer = TfidfVectorizer()
vectors = vectorizer.fit(all_lines)

feature_names = vectorizer.get_feature_names_out()


#tf-idf for each character
for character in characters:
    character_lines = []
    for line in train_set:
        if character != 'others' and character in line['speakers']:
            character_lines.append(line['transcript'])
        elif character == 'others' and line['speakers'][0] not in characters:
            character_lines.append(line['transcript'])


    tf_idf = vectorizer.transform(character_lines)

    dense = tf_idf.todense()
    denselist = dense.tolist()
    df = pd.DataFrame(denselist, columns=feature_names)

    values = []

    for sentence in dense:
        values.append(sentence.sum())

    
    #associate values with sentences
    sentences = []
    for i in range(len(character_lines)):
        sentences.append({'sentence': character_lines[i], 'value': values[i]})
    
    #sort sentences by value
    sentences = sorted(sentences, key = lambda i: i['value'], reverse=True)

    #get highest x sentences
    x = 10
    highest = sentences[:x]

    print(character)

    for sentence in highest:
        print(sentence['sentence'])

    print('\n')


Monica Geller
Y'know, so I don't read as many important books as you do, and I don't write trick poems that seem to be about one thing but are actually about something else. And y'know what, I get excited about stupid stuff, like when I my People magazine comes on Saturday, and the new Hold Everything catalog. Y'know but that does not mean that I'm empty, I care about things. I care about my friends and family. You have no right to make that kind of judgment about me.
Absolutely! Now get out of here you!! Okay, I'm running out of places I can touch him! Look, is there something wrong with me? I mean why am I only attracted to guys where there's no future? Either they're too old, or they're too young, and then there's Pete who's-who's crazy about me, and who's absolutely perfect for me, and there's like zip going on! I mean, seriously, does it sound like something's wrong with me?!
No, look, she's obviously unstable, okay? I mean she's thinking about running out on her wedding day. Okay

In [ ]:
# Example dialogue lines for each character (replace with your actual dataset)
monica_lines = ["I'm going to clean the apartment today.", "I love cooking and organizing."]
ross_lines = ["We were on a break!", "I'm a paleontologist."]
chandler_lines = ["Could I be wearing any more clothes?", "Could this report BE any more boring?"]
joey_lines = ["How you doin'?", "Joey doesn't share food!"]
phoebe_lines = ["Smelly cat, smelly cat, what are they feeding you?", "I found a thumb in my soda can once."]
rachel_lines = ["I'm not so good with the advice. Can I interest you in a sarcastic comment?","I was a shoe!"]

# Combine all dialogue lines into a single list
all_lines = monica_lines + ross_lines + chandler_lines + joey_lines + phoebe_lines + rachel_lines

vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(all_lines)
feature_names = vectorizer.get_feature_names_out()
dense = vectors.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=feature_names)


for v in denselist:
    print(v)

print(df)